In [1]:
%store -r import_all_libs

In [2]:
import_all_libs

In [3]:
import re
from datetime import datetime, date

In [4]:
def artists_to_list(data):
    artists = data
    if isinstance(data, pd.DataFrame):
        artists = data['artists']
    artists = artists.split("']")
    artists.pop() # remove last string thats not an artist
    return [x[x.find(next(filter(str.isalpha, x))):].translate({ord(c): "" for c in "!\'@#$%^&*()[]{};:./<>?\|`~-=_+"}).split(',')[0] for x in artists]


In [5]:
base_folder = 'data'
last_execution = '2022-06-12'
df = pd.read_csv(f'../{base_folder}/df_{last_execution}.csv')

In [6]:
df

,added_at,album_name,track_number,artists,name,popularity,track_id,danceability,energy,key,...,tempo,type,id,uri,track_href,analysis_url,duration_ms,time_signature,playlist_name,playlist_description
0,2022-06-10T03:01:00Z,Fica Até Umas Hora,1,"[['Black Alien', '6aCbXH85qN6xo54C7atSMx']]",Fica Até Umas Hora,45,7xga0gWCTSUpU7Uz5pldQS,0.788,0.591,11,...,74.884,audio_features,7xga0gWCTSUpU7Uz5pldQS,spotify:track:7xga0gWCTSUpU7Uz5pldQS,https://api.spotify.com/v1/tracks/7xga0gWCTSUp...,https://api.spotify.com/v1/audio-analysis/7xga...,228800,4,AULAS,black alien é AULAS
1,2022-06-10T03:01:00Z,The Cypher Deffect 3,1,"[['Costa Gold', '7q1aEytv83jXNECmyaMhgn'], ['T...",The Cypher Deffect 3,62,24THjBgQ5pBt7aipk78ti3,0.574,0.660,1,...,82.330,audio_features,24THjBgQ5pBt7aipk78ti3,spotify:track:24THjBgQ5pBt7aipk78ti3,https://api.spotify.com/v1/tracks/24THjBgQ5pBt...,https://api.spotify.com/v1/audio-analysis/24TH...,276882,4,AULAS,black alien é AULAS
2,2022-06-10T03:01:00Z,Deus É o Meu Guia,1,"[['Chefin', '68PYmgkbRP1qZnEWOry7sB'], ['Mains...",Deus É o Meu Guia,72,4BfppnNi6ZQTI8Ezo97zX6,0.881,0.373,1,...,114.974,audio_features,4BfppnNi6ZQTI8Ezo97zX6,spotify:track:4BfppnNi6ZQTI8Ezo97zX6,https://api.spotify.com/v1/tracks/4BfppnNi6ZQT...,https://api.spotify.com/v1/audio-analysis/4Bfp...,166957,4,AULAS,black alien é AULAS
3,2022-06-10T03:01:00Z,Coisas Que Eu Só Falaria Brincando,7,"[['Sid', '05BrFNfdik79RdmOG1Ux1Q'], ['NOG', '1...",Sitio do Tio Harry,65,0a3Bfmq7czUFUsk2VZUA28,0.682,0.694,1,...,84.306,audio_features,0a3Bfmq7czUFUsk2VZUA28,spotify:track:0a3Bfmq7czUFUsk2VZUA28,https://api.spotify.com/v1/tracks/0a3Bfmq7czUF...,https://api.spotify.com/v1/audio-analysis/0a3B...,291882,4,AULAS,black alien é AULAS
4,2022-06-10T03:01:00Z,Sigo na Fé do Pai,1,"[['Salvador Da Rima', '3zUcyANWSbo98ikca4ugrV'...",Sigo na Fé do Pai,66,7zcQ56JRoNd4JAwGKcrxiZ,0.499,0.762,6,...,173.262,audio_features,7zcQ56JRoNd4JAwGKcrxiZ,spotify:track:7zcQ56JRoNd4JAwGKcrxiZ,https://api.spotify.com/v1/tracks/7zcQ56JRoNd4...,https://api.spotify.com/v1/audio-analysis/7zcQ...,264828,4,AULAS,black alien é AULAS
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1224,2022-02-25T03:01:00Z,A Invasão do Sagaz Homem Fumaça,8,"[['Planet Hemp', '3nasa4yabzvUeb6DzejDYU']]",Contexto,50,5eS43KfdjIfUWxAXpfXT8x,0.780,0.869,1,...,87.663,audio_features,5eS43KfdjIfUWxAXpfXT8x,spotify:track:5eS43KfdjIfUWxAXpfXT8x,https://api.spotify.com/v1/tracks/5eS43KfdjIfU...,https://api.spotify.com/v1/audio-analysis/5eS4...,225293,4,No Princípio Era o Verso,Uma playlist só com as origens do Rap nacional...
1225,2022-02-25T03:01:00Z,O Lado B do Hip Hop,5,"[['Sp Funk', '6gOXHHfszKFFvIaKUe2aTN'], ['RZO'...",Enxame,28,21HGlwk8A6uiTMdb74W4Jb,0.734,0.889,6,...,85.165,audio_features,21HGlwk8A6uiTMdb74W4Jb,spotify:track:21HGlwk8A6uiTMdb74W4Jb,https://api.spotify.com/v1/tracks/21HGlwk8A6ui...,https://api.spotify.com/v1/audio-analysis/21HG...,391400,4,No Princípio Era o Verso,Uma playlist só com as origens do Rap nacional...
1226,2022-02-25T03:01:00Z,Valeu a Experiência,2,"[['Filosofia de Rua', '7rwCXuDiu5DR1oLOmTBOLz'...",Se o Mundo Inteiro Pudesse Me Ouvir,32,4pEi1lS6zX3MSsMUH1BBwz,0.686,0.855,1,...,155.838,audio_features,4pEi1lS6zX3MSsMUH1BBwz,spotify:track:4pEi1lS6zX3MSsMUH1BBwz,https://api.spotify.com/v1/tracks/4pEi1lS6zX3M...,https://api.spotify.com/v1/audio-analysis/4pEi...,336533,4,No Princípio Era o Verso,Uma playlist só com as origens do Rap nacional...
1227,2022-02-25T03:01:00Z,Provérbios 13,10,"[['509-E', '1PA11Cy0SrjmwymvGS9h0C']]",Saudades Mil,0,79zU8jvVMf5GEvpFqBF1Ya,0.613,0.589,2,...,76.739,audio_features,79zU8jvVMf5GEvpFqBF1Ya,spotify:track:79zU8jvVMf5GEvpFqBF1Ya,https://api.spotify.com/v1/tracks/79zU8jvVMf5G...,https://api.spotify.com/v1/audio-analysis/79zU...,526027,4,No Princípio Era o Verso,Uma playlist só com as origens do Rap nacional...


## Fields description

### popularity, integer

The popularity of the artist. The value will be between 0 and 100, with 100 being the most popular. The artist's popularity is calculated from the popularity of all the artist's tracks.

### duration_ms, integer

The track length in milliseconds.

### acousticness, number\<float\>

A confidence measure from 0.0 to 1.0 of whether the track is acoustic. 1.0 represents high confidence the track is acoustic. >= 0 <= 1
       
### danceability, number\<float\>

Danceability describes how suitable a track is for dancing based on a combination of musical elements including tempo, rhythm stability, beat strength, and overall regularity. A value of 0.0 is least danceable and 1.0 is most danceable.
    
### energy, number\<float\>

Energy is a measure from 0.0 to 1.0 and represents a perceptual measure of intensity and activity. Typically, energetic tracks feel fast, loud, and noisy. For example, death metal has high energy, while a Bach prelude scores low on the scale. Perceptual features contributing to this attribute include dynamic range, perceived loudness, timbre, onset rate, and general entropy.
    
### instrumentalness, number\<float\>

Predicts whether a track contains no vocals. "Ooh" and "aah" sounds are treated as instrumental in this context. Rap or spoken word tracks are clearly "vocal". The closer the instrumentalness value is to 1.0, the greater likelihood the track contains no vocal content. Values above 0.5 are intended to represent instrumental tracks, but confidence is higher as the value approaches 1.0.
     
### key, integer

The key the track is in. Integers map to pitches using standard Pitch Class notation. E.g. 0 = C, 1 = C♯/D♭, 2 = D, and so on. If no key was detected, the value is -1. >= -1 <= 11
                 
### liveness, number\<float\>

Detects the presence of an audience in the recording. Higher liveness values represent an increased probability that the track was performed live. A value above 0.8 provides strong likelihood that the track is live.

### loudness, number\<float\>

The overall loudness of a track in decibels (dB). Loudness values are averaged across the entire track and are useful for comparing relative loudness of tracks. Loudness is the quality of a sound that is the primary psychological correlate of physical strength (amplitude). Values typically range between -60 and 0 db.
    
### mode, integer

Mode indicates the modality (major or minor) of a track, the type of scale from which its melodic content is derived. Major is represented by 1 and minor is 0.
    
### speechiness, number\<float\>

Speechiness detects the presence of spoken words in a track. The more exclusively speech-like the recording (e.g. talk show, audio book, poetry), the closer to 1.0 the attribute value. Values above 0.66 describe tracks that are probably made entirely of spoken words. Values between 0.33 and 0.66 describe tracks that may contain both music and speech, either in sections or layered, including such cases as rap music. Values below 0.33 most likely represent music and other non-speech-like tracks.
    
### tempo, number\<float\>

The overall estimated tempo of a track in beats per minute (BPM). In musical terminology, tempo is the speed or pace of a given piece and derives directly from the average beat duration.
    
### time_signature, integer

An estimated time signature. The time signature (meter) is a notational convention to specify how many beats are in each bar (or measure). The time signature ranges from 3 to 7 indicating time signatures of "3/4", to "7/4". >= 3 <= 7
            
### valence, number\<float\>

A measure from 0.0 to 1.0 describing the musical positiveness conveyed by a track. Tracks with high valence sound more positive (e.g. happy, cheerful, euphoric), while tracks with low valence sound more negative (e.g. sad, depressed, angry). >= 0 <= 1

### valence, number\<float\>

A measure from 0.0 to 1.0 describing the musical positiveness conveyed by a track. Tracks with high valence sound more positive (e.g. happy, cheerful, euphoric), while tracks with low valence sound more negative (e.g. sad, depressed, angry). >= 0 <= 1.

In [7]:
df.columns

Index(['added_at', 'album_name', 'track_number', 'artists', 'name',
       'popularity', 'track_id', 'danceability', 'energy', 'key', 'loudness',
       'mode', 'speechiness', 'acousticness', 'instrumentalness', 'liveness',
       'valence', 'tempo', 'type', 'id', 'uri', 'track_href', 'analysis_url',
       'duration_ms', 'time_signature', 'playlist_name',
       'playlist_description'],
      dtype='object')

In [8]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1229 entries, 0 to 1228
Data columns (total 27 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   added_at              1229 non-null   object 
 1   album_name            1229 non-null   object 
 2   track_number          1229 non-null   int64  
 3   artists               1229 non-null   object 
 4   name                  1229 non-null   object 
 5   popularity            1229 non-null   int64  
 6   track_id              1229 non-null   object 
 7   danceability          1229 non-null   float64
 8   energy                1229 non-null   float64
 9   key                   1229 non-null   int64  
 10  loudness              1229 non-null   float64
 11  mode                  1229 non-null   int64  
 12  speechiness           1229 non-null   float64
 13  acousticness          1229 non-null   float64
 14  instrumentalness      1229 non-null   float64
 15  liveness             

In [9]:
df.shape

(1229, 27)

In [10]:
df.added_at = pd.to_datetime(df.added_at)

In [11]:
df = df[~df.duplicated(subset=['name'])] # removes duplicated songs

In [12]:
df.columns

Index(['added_at', 'album_name', 'track_number', 'artists', 'name',
       'popularity', 'track_id', 'danceability', 'energy', 'key', 'loudness',
       'mode', 'speechiness', 'acousticness', 'instrumentalness', 'liveness',
       'valence', 'tempo', 'type', 'id', 'uri', 'track_href', 'analysis_url',
       'duration_ms', 'time_signature', 'playlist_name',
       'playlist_description'],
      dtype='object')

In [13]:
drop_columns = [
    'track_id',
    'uri',
    'id',
    'track_href',
    'analysis_url',
    'type' # contains only audio_features
]
df = df.drop(columns=drop_columns, axis=1)

In [14]:
df.columns

Index(['added_at', 'album_name', 'track_number', 'artists', 'name',
       'popularity', 'danceability', 'energy', 'key', 'loudness', 'mode',
       'speechiness', 'acousticness', 'instrumentalness', 'liveness',
       'valence', 'tempo', 'duration_ms', 'time_signature', 'playlist_name',
       'playlist_description'],
      dtype='object')

In [15]:
br_playlists = [
    'Presença Hip Hop', 
    'Rap Acústico',
    'No Princípio Era o Verso',
    'O Drip',
    'AMPLIFIKA',
    'O Melhor da Trapperz Brasil 2021',
    'Copão na mão',
    'AULAS',
    'Dose Trap', 
    'Trap Ouro', 
    'creme', 
    'Gigantes do Rap Nacional', 
    'TRAPando', 
    'Trapperz Brasil', 
    'Trap Classe', 
    'Flow Romance', 
    'Resenha na Praia', 
    'RAPública']

df_brazil = df[df.playlist_name.isin(br_playlists)]

In [16]:
df_brazil.playlist_name.nunique() < len(br_playlists)

True

In [17]:
df.artists = df.artists.apply(artists_to_list)

In [18]:
df.head()

,added_at,album_name,track_number,artists,name,popularity,danceability,energy,key,loudness,...,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration_ms,time_signature,playlist_name,playlist_description
0,2022-06-10 03:01:00+00:00,Fica Até Umas Hora,1,[Black Alien],Fica Até Umas Hora,45,0.788,0.591,11,-7.744,...,0.3580,0.622,0.000053,0.3620,0.712,74.884,228800,4,AULAS,black alien é AULAS
1,2022-06-10 03:01:00+00:00,The Cypher Deffect 3,1,"[Costa Gold, Tz da Coronel, Major RD, André Nine]",The Cypher Deffect 3,62,0.574,0.660,1,-6.694,...,0.3890,0.299,0.000000,0.1140,0.667,82.330,276882,4,AULAS,black alien é AULAS
2,2022-06-10 03:01:00+00:00,Deus É o Meu Guia,1,"[Chefin, Mainstreet]",Deus É o Meu Guia,72,0.881,0.373,1,-8.908,...,0.0582,0.215,0.000000,0.0770,0.909,114.974,166957,4,AULAS,black alien é AULAS
3,2022-06-10 03:01:00+00:00,Coisas Que Eu Só Falaria Brincando,7,"[Sid, NOG, André Nine, Chiocki]",Sitio do Tio Harry,65,0.682,0.694,1,-6.353,...,0.4300,0.448,0.000000,0.2980,0.732,84.306,291882,4,AULAS,black alien é AULAS
4,2022-06-10 03:01:00+00:00,Sigo na Fé do Pai,1,"[Salvador Da Rima, Mc Poze do Rodo, MC Ryan SP...",Sigo na Fé do Pai,66,0.499,0.762,6,-4.163,...,0.4530,0.141,0.000000,0.0988,0.622,173.262,264828,4,AULAS,black alien é AULAS


In [19]:
df_brazil.to_csv(f'../{base_folder}/df_brazil_{last_execution}.csv', index=False)

In [20]:
df.to_csv(f'../{base_folder}/df_processed_{last_execution}.csv', index=False)

In [21]:
df_brazil.shape

(716, 21)

In [22]:
df.shape

(975, 21)